In [1]:
# imports 
import numpy as np
import pandas as pd

In [2]:
df= pd.read_csv('../data/train/diamonds_train.csv', index_col=[0])
df.head().T

,0,1,2,3,4
index_id,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,41667f6e2629360aecaf00b20f8732e3310417ebd54b24...,01f8667f50d52677bea23231a74156e4f92360d7bc3db6...,c3867352aab641358faec75d733af012dbe2259a014ea8...,0da4b104c4d8589fcb96a03aa0787549a2631935b0f499...
depth,62.4,61.6,62.3,59.6,60.2
table,58.0,58.0,58.0,60.0,62.0
x,6.83,6.4,5.86,7.58,5.4
y,6.79,6.35,5.8,7.48,5.33
z,4.25,3.93,3.63,4.49,3.23
price,4268,3513,1792,7553,1176
carat,1.21,1.02,0.77,1.51,0.57
cut,Premium,Premium,Premium,Premium,Premium
color,J,J,J,J,J


In [3]:
numerical_columns = ['carat', 'depth', 'table', 'x', 'y', 'z']
categorical_columns = ['cut', 'color', 'clarity']
FEATS = numerical_columns + categorical_columns
TARGET = 'price'


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder



categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])


In [5]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])



In [6]:
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numerical_columns),
                                                ('cat', categorical_transformer, categorical_columns)])

In [7]:
pd.DataFrame(data=preprocessor.fit_transform(df)).head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.867006,0.452019,0.247981,0.978807,0.921985,1.022657,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.467458,-0.106755,0.247981,0.596394,0.538254,0.563582,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.058262,0.382172,0.247981,0.116154,0.058590,0.133198,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.497870,-1.503688,1.143433,1.645806,1.523746,1.366964,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.478838,-1.084608,2.038886,-0.292939,-0.351305,-0.440646,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[FEATS], df[TARGET], test_size=0.15, random_state=42);

In [9]:
from xgboost import XGBRegressor


model= Pipeline(steps=[('preprocessor', preprocessor),
                       ("rf_classifier",XGBRegressor())])

/Users/ivan.repilado/miniconda3/envs/sklearn-env/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [10]:
model.fit(X_train, y_train);

In [11]:
y_pred = model.predict(X_test)
y_pred

array([8484.328  ,  492.47852, 8066.5923 , ..., 1324.2136 ,  939.4298 ,
       1600.7781 ], dtype=float32)

In [12]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred)**0.5
rmse

612.4052165844137

Tal y como vemos, el RMSE es mas elevado que con el modelo XGBRegressor sin escalar, por lo que descartamos escalar en este modelo